### Load libs

In [1]:
import pandas as pd
import os
import re
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
n = 750
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Combine files

In [3]:
term = 'AmeriCredit Automobile Receivables Trust'

In [4]:
folder = 'data/transaction/prepared/'
files = [folder + file for file in os.listdir(folder) if '.csv' in file and term in file and '._' not in file]

In [5]:
files.sort()
files

['data/transaction/AmeriCredit Automobile Receivables Trust 2017-1 Data Tape transaction prepared.csv',
 'data/transaction/AmeriCredit Automobile Receivables Trust 2017-2 Data Tape transaction prepared.csv',
 'data/transaction/AmeriCredit Automobile Receivables Trust 2017-3 Data Tape transaction prepared.csv',
 'data/transaction/AmeriCredit Automobile Receivables Trust 2017-4 Data Tape transaction prepared.csv',
 'data/transaction/AmeriCredit Automobile Receivables Trust 2019-1 Data Tape transaction prepared.csv']

In [6]:
finder = re.compile('\d{4,}\W\d{1,}')

In [7]:
master = pd.DataFrame()
for file in files:
    add_id = re.findall(finder, file)[0]
    df = pd.read_csv(file)
    df['securitization'] = add_id
    print(df.shape)
    master = master.append(df).reset_index(drop = True)

/anaconda3/envs/kn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1509167, 101)
(2053468, 101)
(1738915, 101)
(1875238, 101)
(1141664, 101)


In [8]:
master.shape

(8318452, 101)

In [9]:
master['target'].value_counts(dropna = False)

Active or other    4789312
Prepaid            2074203
Charged-off        1093425
Closed             361512 
Name: target, dtype: int64

### Fix vals

In [10]:
vals_cols = [col for col in list(master.columns) if 'vals' in col.lower()]

In [11]:
def fix_vals(row, column):

    """
    Fix column values
    """

    init = str(row[column])

    ret_val = 'str: ' + init

    return ret_val


In [12]:
for col in vals_cols:
    print(col)
    master[col] = master[col].astype(str)
    master[col] = 'str: ' + master[col]
    

In [13]:
master.shape

(8318452, 101)

### Rename id

In [14]:
master.rename(columns = {'ID': 'idOld'}, inplace = True)

### Export

In [17]:
e_folder = folder + 'combined/'
e_file = '{} securitizations transaction.csv'.format(term)
e_path = e_folder + e_file
e_path

'data/transaction/combined/AmeriCredit Automobile Receivables Trust securitizations transaction.csv'

In [18]:
master.to_csv(e_path, index = False)

In [19]:
print('complete...')

complete...


### End